In [110]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 100)

In [111]:
dataset = pd.read_csv("data/mostrecent_data.csv")
dataset.head(100)

,NbId,NbName,DistrictId,DistrictName,Geoshape,GeoshapePerimeter,GeoshapeArea,GeoshapeCenter,number of residents,ScoreDiversity,NumberHouseholds,AvgHouseValue,PctComplainsAQ,PctComplainsNoise,AvgElectricityUsage,PctUnhappy,PctHighEducation,PctLowEducation,PctFeelsUnsafe,ScoreSocialCohesion,ScoreGoodLife,PctUnemployed,PctEconomicallyIndependent,AvgIncome,NumberShops,Distance2Childcare,Distance2Trainstation,Distance2HighwayEntrypoint,Distance2FamilyDoctor,Distance2Cinema,Distance2Cafe,Distance2SwimmingPool
0,546,Bokt,5,Stadsdeel Woensel-Noord,"{""coordinates"": [[[5.496376105740801, 51.49677...",4850.853028,1.394571e+06,"51.491628626659654, 5.504372699789956",128.0,.,45,.,.,.,4570,.,-,-,.,.,.,-,.,.,0.0,"1,6","6,5","1,9","1,9","7,2","0,9","4,8"
1,426,Vredeoord,4,Stadsdeel Woensel-Zuid,"{""coordinates"": [[[5.4478878937157615, 51.4684...",5030.135514,1.343777e+06,"51.4656799043364, 5.443837232394569",477.0,.,65,.,.,.,4670,.,"0,0","83,3",.,.,.,37,.,"9,6",0.0,"1,6","2,6","3,0","1,3","3,6","1,7","1,9"
2,115,TU-terrein,1,Stadsdeel Centrum,"{""coordinates"": [[[5.480378012450603, 51.44638...",4552.423442,1.240211e+06,"51.447493765789126, 5.4905343160054185",760.0,"0,89",705,111,.,.,850,.,"64,5","2,6",.,.,.,3,"79,4","24,0",1.0,"0,2","1,6","3,0","1,7","2,2","1,5","5,0"
3,632,Meerbos,6,Stadsdeel Strijp,"{""coordinates"": [[[5.4207997480240495, 51.4440...",5813.572286,1.233203e+06,"51.45056114369441, 5.4168505397948685",54.0,.,30,.,.,.,.,.,-,-,.,.,.,50,.,.,0.0,"1,6","3,7","1,6","2,0","3,0","0,9","4,4"
4,532,Tempel,5,Stadsdeel Woensel-Noord,"{""coordinates"": [[[5.469716276375012, 51.48407...",4525.443091,1.190267e+06,"51.48029692517017, 5.479872107812958",5179.0,"0,57",2395,255,22,33,2540,5,"26,0","32,7",13,"5,9","7,3",8,"68,5","39,3",25.0,"0,6","4,6","1,6","0,5","5,2","0,5","2,5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,722,Blaarthem,7,Stadsdeel Gestel,"{""coordinates"": [[[5.462739127078532, 51.42793...",3055.620291,3.619271e+05,"51.42435886349648, 5.457445971309711",2526.0,"0,65",1400,232,25,50,2240,10,"32,5","35,0",24,"5,5","6,9",12,"63,7","33,8",10.0,"0,6","3,7","2,2","0,6","2,7","0,3","2,2"
96,616,Strijp S,6,Stadsdeel Strijp,"{""coordinates"": [[[5.4611007030010486, 51.4435...",2222.100024,2.983105e+05,"51.446633587850286, 5.457720010664559",3010.0,"0,43",2015,275,30,65,1670,9,"56,2","19,9",5,"6,1","8,0",6,"87,9","38,6",35.0,"1,1","1,0","2,2","1,1","0,6","0,2","4,3"
97,713,Hagenkamp,7,Stadsdeel Gestel,"{""coordinates"": [[[5.457194923998502, 51.43013...",2355.595966,2.729910e+05,"51.431951756726, 5.460750863019136",1201.0,"0,57",770,219,25,49,1950,8,"34,6","32,1",7,"6,4","7,9",9,"69,9","30,3",1.0,"0,3","2,9","2,5","0,4","2,0","0,7","3,1"
98,712,Oude Spoorbaan,7,Stadsdeel Gestel,"{""coordinates"": [[[5.471122635535266, 51.43315...",2471.289285,2.663381e+05,"51.4313155428131, 5.46646737715935",2204.0,"0,64",1380,240,25,49,2080,8,"57,6","17,3",7,"6,4","7,9",5,"77,1","37,8",11.0,"0,4","2,6","3,2","0,6","1,6","0,4","2,9"


In [112]:
dataset.drop("ScoreDiversity", axis=1, inplace=True)
dataset.drop("PctEconomicallyIndependent", axis=1, inplace=True)
dataset.drop("GeoshapeCenter", axis=1, inplace=True)
dataset.drop("NumberHouseholds", axis=1, inplace=True)
dataset.drop("PctFeelsUnsafe", axis=1, inplace=True)
dataset.drop("AvgElectricityUsage", axis=1, inplace=True)
dataset.drop("PctComplainsAQ", axis=1, inplace=True)
dataset.drop("PctLowEducation", axis=1, inplace=True)
dataset.drop("AvgHouseValue", axis=1, inplace=True)

In [113]:
dataset['Distance2Trainstation'] = dataset['Distance2Trainstation'].replace('.', 1000)
dataset['Distance2HighwayEntrypoint'] = dataset['Distance2HighwayEntrypoint'].replace('.', 1000)
dataset['Distance2Trainstation'] = dataset['Distance2Trainstation'].str.replace(',', '.')
dataset['Distance2HighwayEntrypoint'] = dataset['Distance2HighwayEntrypoint'].str.replace(',', '.')
dataset['Distance2Trainstation'] = dataset['Distance2Trainstation'].fillna(1000)
dataset['Distance2HighwayEntrypoint'] = dataset['Distance2HighwayEntrypoint'].fillna(1000)


dataset['PctHighEducation'] = dataset['PctHighEducation'].fillna("-1")
dataset['AvgIncome'] = dataset['AvgIncome'].fillna("-1")
dataset['ScoreGoodLife'] = dataset['ScoreGoodLife'].fillna("-1")

dataset["Distance2Trainstation"] = pd.to_numeric(dataset['Distance2Trainstation'], errors='coerce')
dataset["Distance2HighwayEntrypoint"] = pd.to_numeric(dataset['Distance2HighwayEntrypoint'], errors='coerce')

dataset['new feature'] = pd.to_numeric(dataset['Shops per person'], errors='coerce')

# Identify rows with non-convertible values
non_convertible_rows = dataset[dataset['new feature'].isna()]

# Extract the original non-convertible values
non_convertible_values = non_convertible_rows['Shops per person']

# Print or store the non-convertible values
print("Non-convertible values:", non_convertible_values.tolist())

In [114]:
dataset['Distance2TrainstationR'] = dataset['Distance2Trainstation'].rank(method='first', ascending=True).astype(int)
dataset['Distance2HighwayEntrypointR'] = dataset['Distance2HighwayEntrypoint'].rank(method='first', ascending=True).astype(int)
dataset['Infrastructure availability'] = dataset[['Distance2HighwayEntrypointR', 'Distance2TrainstationR']].mean(axis=1).rank(method='first').astype(int)
dataset.drop("Distance2HighwayEntrypointR", axis=1, inplace=True)
dataset.drop("Distance2TrainstationR", axis=1, inplace=True)

In [115]:
dataset = dataset.sort_values(by='Infrastructure availability', ascending=True)

In [116]:
dataset.head(15)

,NbId,NbName,DistrictId,DistrictName,Geoshape,GeoshapePerimeter,GeoshapeArea,number of residents,PctComplainsNoise,PctUnhappy,PctHighEducation,ScoreSocialCohesion,ScoreGoodLife,PctUnemployed,AvgIncome,NumberShops,Distance2Childcare,Distance2Trainstation,Distance2HighwayEntrypoint,Distance2FamilyDoctor,Distance2Cinema,Distance2Cafe,Distance2SwimmingPool,Infrastructure availability
81,624,Drents Dorp,6,Stadsdeel Strijp,"{""coordinates"": [[[5.448604988371642, 51.44490...",3173.128722,4.462157e+05,2420.0,47,10,"23,1","5,9","7,4",12,"33,7",0.0,"0,5",1.7,1.6,"0,9","1,1","0,4","4,4",1
36,615,Schoot,6,Stadsdeel Strijp,"{""coordinates"": [[[5.448604988371642, 51.44490...",2848.360841,3.871639e+05,3547.0,52,7,"63,3","6,2","7,7",4,"37,2",18.0,"0,7",1.5,1.9,"0,7","0,6","0,3","4,6",2
96,616,Strijp S,6,Stadsdeel Strijp,"{""coordinates"": [[[5.4611007030010486, 51.4435...",2222.100024,2.983105e+05,3010.0,65,9,"56,2","6,1","8,0",6,"38,6",35.0,"1,1",1.0,2.2,"1,1","0,6","0,2","4,3",3
10,622,Het Ven,6,Stadsdeel Strijp,"{""coordinates"": [[[5.448934970007217, 51.44381...",3981.243617,8.808791e+05,3822.0,52,7,"36,7","6,2","7,7",8,"37,7",22.0,"0,5",2.5,1.7,"0,9","1,7","0,4","4,3",4
93,336,Karpen,3,Stadsdeel Tongelre,"{""coordinates"": [[[5.49829999177369, 51.457664...",3794.553430,9.280669e+05,475.0,53,4,"73,3","6,5","8,1",-,"150,2",1.0,"1,0",2.8,1.5,"1,3","3,4","1,2","4,4",5
54,623,Lievendaal,6,Stadsdeel Strijp,"{""coordinates"": [[[5.4381275411052385, 51.4415...",4886.052554,8.869529e+05,3152.0,43,6,"25,3","5,7","7,3",8,"37,7",3.0,"0,5",2.6,1.7,"0,9","1,8","0,3","4,3",6
107,625,Zwaanstraat,6,Stadsdeel Strijp,"{""coordinates"": [[[5.451767570020675, 51.44794...",3249.677428,5.249004e+05,1367.0,40,1,"83,9","7,4","8,4",2,"72,3",0.0,"0,7",1.6,2.2,"1,5","1,1","0,6","3,8",7
13,621,Hurk,6,Stadsdeel Strijp,"{""coordinates"": [[[5.454409513125079, 51.43180...",6401.073430,2.077451e+06,51.0,.,.,"16,7",.,.,-,.,29.0,"1,0",3.2,1.3,"1,5","2,3","0,6","3,8",8
56,626,Wielewaal,6,Stadsdeel Strijp,"{""coordinates"": [[[5.4250222108687725, 51.4614...",6491.113711,2.323453e+06,110.0,.,.,"60,0",.,.,-,.,0.0,"1,0",3.3,1.4,"2,8","3,5","2,1","3,8",9
48,335,'t Hofke,3,Stadsdeel Tongelre,"{""coordinates"": [[[5.520527534552106, 51.44707...",6304.804603,1.503983e+06,3805.0,45,6,"30,7","6,3","7,5",8,"39,3",6.0,"0,5",3.6,1.0,"0,9","4,0","0,7","3,9",10


In [117]:
dataset['Distance2SwimmingPool'] = dataset['Distance2SwimmingPool'].str.replace(',', '.')
dataset['Distance2Cinema'] = dataset['Distance2Cinema'].str.replace(',', '.')
dataset['Distance2Cafe'] = dataset['Distance2Cafe'].str.replace(',', '.')
dataset['Distance2Сafe'] = dataset['Distance2Cafe'].replace('.', 1000)
dataset['Distance2Cinema'] = dataset['Distance2Cinema'].replace('.', 1000)
dataset['Distance2SwimmingPool'] = dataset['Distance2SwimmingPool'].replace('.', 1000)
dataset["Distance2Cinema"] = dataset["Distance2Cinema"].fillna("1000.0")
dataset["Distance2Cafe"] = dataset["Distance2Cafe"].fillna(1000.0)
dataset["Distance2SwimmingPool"] = dataset["Distance2SwimmingPool"].fillna(1000)
dataset["Distance2SwimmingPool"] = dataset["Distance2SwimmingPool"].fillna(1000)
dataset["NumberShops"] = dataset["NumberShops"].fillna(1000)


In [118]:
dataset["Distance2Cafe"] = pd.to_numeric(dataset['Distance2Cafe'], errors='coerce')
dataset["Distance2SwimmingPool"] = pd.to_numeric(dataset['Distance2SwimmingPool'], errors='coerce')
dataset["Distance2Cinema"] = pd.to_numeric(dataset['Distance2Cinema'], errors='coerce')
dataset = dataset.fillna(1000)
dataset['Distance2Cafe'] = dataset['Distance2Cafe'].rank(method='first', ascending=True).astype(int)
dataset['Distance2Cinema'] = dataset['Distance2Cinema'].rank(method='first', ascending=True).astype(int)
dataset['Distance2SwimmingPool'] = dataset['Distance2SwimmingPool'].rank(method='first', ascending=True).astype(int)
dataset['Shops and entertainment'] = dataset[['Distance2Cinema', 'Distance2Cafe', 'Distance2SwimmingPool']].mean(axis=1).rank(method='first').astype(int)
dataset.drop('Distance2Cafe', axis=1, inplace=True)
dataset.drop('Distance2SwimmingPool', axis=1, inplace=True)
dataset.drop('Distance2Cinema', axis=1, inplace=True)

In [119]:
dataset = dataset.sort_values(by='Shops and entertainment', ascending=True)
dataset["Shops per area"] = dataset['NumberShops'] / dataset['GeoshapeArea'] 
dataset["Shops per person"] = dataset['NumberShops'] / dataset['number of residents'] 
dataset['Shops per area'] = dataset['Shops per area'].rank(method='first', ascending=False).astype(int)
dataset['Shops per person'] = dataset['Shops per person'].rank(method='first', ascending=False).astype(int)
dataset['Final rank shops'] = dataset[['Shops per area', 'Shops per person']].mean(axis=1).rank(method='first').astype(int)
dataset['Shops and entertainment'] = dataset[['Shops and entertainment', 'Final rank shops']].mean(axis=1).rank(method='first').astype(int)
dataset.drop("Shops per area", axis=1, inplace=True)
dataset.drop("Shops per person", axis=1, inplace=True)
dataset.drop("Final rank shops", axis=1, inplace=True)

In [120]:
dataset = dataset.sort_values(by='Shops and entertainment', ascending=True)
dataset.head()

,NbId,NbName,DistrictId,DistrictName,Geoshape,GeoshapePerimeter,GeoshapeArea,number of residents,PctComplainsNoise,PctUnhappy,PctHighEducation,ScoreSocialCohesion,ScoreGoodLife,PctUnemployed,AvgIncome,NumberShops,Distance2Childcare,Distance2Trainstation,Distance2HighwayEntrypoint,Distance2FamilyDoctor,Infrastructure availability,Distance2Сafe,Shops and entertainment
45,216,Bloemenplein,2,Stadsdeel Stratum,"{""coordinates"": [[[5.487913334889891, 51.42373...",1695.060044,133534.195474,1258.0,51,9,"36,6","6,0","7,2",8,"36,0",23.0,"0,9",2.9,2.5,"0,8",25,0.3,1
86,236,Nieuwe Erven,2,Stadsdeel Stratum,"{""coordinates"": [[[5.492793179430379, 51.42417...",1635.447970,151145.626948,1136.0,44,9,"31,3","6,4","8,0",10,"31,6",22.0,"0,3",3.6,2.2,"0,6",40,0.2,2
16,112,Bergen,1,Stadsdeel Centrum,"{""coordinates"": [[[5.473622298188002, 51.43419...",2993.849315,346126.649183,2832.0,72,6,"61,8","5,5","7,6",3,"45,8",66.0,"0,5",2.2,3.7,"0,5",68,0.2,3
71,111,Binnenstad,1,Stadsdeel Centrum,"{""coordinates"": [[[5.486100329477308, 51.43455...",3583.765529,665717.709441,4108.0,76,5,"56,8","5,4","7,5",7,"40,3",272.0,"0,9",1.7,3.8,"1,1",74,0.2,4
87,215,Joriskwartier,2,Stadsdeel Stratum,"{""coordinates"": [[[5.4867601412710165, 51.4287...",2008.871505,148966.813518,1275.0,64,6,"43,4","6,1","7,7",6,"40,7",13.0,"0,9",2.9,2.7,"0,9",47,0.3,5


In [121]:
age_df = pd.read_csv("data/extra_dataset_tim_age.csv", sep=";")
age_df = age_df.fillna(-100)
age_df = age_df.replace('x', -100)
age_df['Totaal 0-9 jaar|2023'] = pd.to_numeric(age_df['Totaal 0-9 jaar|2023'], errors='coerce')
age_df['Totaal 10-19 jaar|2023'] = pd.to_numeric(age_df['Totaal 10-19 jaar|2023'], errors='coerce')
age_df['Totaal 20-29 jaar|2023'] = pd.to_numeric(age_df['Totaal 20-29 jaar|2023'], errors='coerce')
age_df['Totaal 30-39 jaar|2023'] = pd.to_numeric(age_df['Totaal 30-39 jaar|2023'], errors='coerce')
age_df['Totaal 40-49 jaar|2023'] = pd.to_numeric(age_df['Totaal 40-49 jaar|2023'], errors='coerce')
age_df['Totaal 50-59 jaar|2023'] = pd.to_numeric(age_df['Totaal 50-59 jaar|2023'], errors='coerce')
age_df['Totaal 60-69 jaar|2023'] = pd.to_numeric(age_df['Totaal 60-69 jaar|2023'], errors='coerce')
age_df['Totaal 70-79 jaar|2023'] = pd.to_numeric(age_df['Totaal 70-79 jaar|2023'], errors='coerce')
age_df['Totaal 80-89 jaar|2023'] = pd.to_numeric(age_df['Totaal 80-89 jaar|2023'], errors='coerce')
age_df["Totaal 90 jaar en ouder|2023"] = pd.to_numeric(age_df["Totaal 90 jaar en ouder|2023"], errors='coerce')


In [122]:
first = age_df["Totaal 0-9 jaar|2023"] * 5 + age_df["Totaal 10-19 jaar|2023"] * 15 + age_df["Totaal 20-29 jaar|2023"] * 25 + age_df["Totaal 30-39 jaar|2023"] * 35 + age_df["Totaal 40-49 jaar|2023"] * 45 + age_df["Totaal 50-59 jaar|2023"] * 55 + age_df["Totaal 60-69 jaar|2023"] * 65 + age_df["Totaal 70-79 jaar|2023"] * 75 + age_df["Totaal 80-89 jaar|2023"] * 85 + age_df["Totaal 90 jaar en ouder|2023"] * 96
second = age_df["Totaal 0-9 jaar|2023"] + age_df["Totaal 10-19 jaar|2023"] + age_df["Totaal 20-29 jaar|2023"] + age_df["Totaal 30-39 jaar|2023"] + age_df["Totaal 40-49 jaar|2023"] + age_df["Totaal 50-59 jaar|2023"] + age_df["Totaal 60-69 jaar|2023"] + age_df["Totaal 70-79 jaar|2023"] + age_df["Totaal 80-89 jaar|2023"] + age_df["Totaal 90 jaar en ouder|2023"]
first = first.apply(lambda item: 10 if item < 0 else item)
age_df["average age"] = first / second
age_df["average age"] = age_df["average age"].round(2)
age_df["average age"] = age_df["average age"].apply(lambda item: "N/A" if item < 5 else item)
age_df = age_df[["Buurten", "average age"]]
age_df.rename(columns={'Buurten': 'NbName'}, inplace=True)
dataset = pd.merge(dataset, age_df, on='NbName', how='inner')
dataset.head()

,NbId,NbName,DistrictId,DistrictName,Geoshape,GeoshapePerimeter,GeoshapeArea,number of residents,PctComplainsNoise,PctUnhappy,PctHighEducation,ScoreSocialCohesion,ScoreGoodLife,PctUnemployed,AvgIncome,NumberShops,Distance2Childcare,Distance2Trainstation,Distance2HighwayEntrypoint,Distance2FamilyDoctor,Infrastructure availability,Distance2Сafe,Shops and entertainment,average age
0,216,Bloemenplein,2,Stadsdeel Stratum,"{""coordinates"": [[[5.487913334889891, 51.42373...",1695.060044,133534.195474,1258.0,51,9,"36,6","6,0","7,2",8,"36,0",23.0,"0,9",2.9,2.5,"0,8",25,0.3,1,37.42
1,236,Nieuwe Erven,2,Stadsdeel Stratum,"{""coordinates"": [[[5.492793179430379, 51.42417...",1635.447970,151145.626948,1136.0,44,9,"31,3","6,4","8,0",10,"31,6",22.0,"0,3",3.6,2.2,"0,6",40,0.2,2,39.26
2,112,Bergen,1,Stadsdeel Centrum,"{""coordinates"": [[[5.473622298188002, 51.43419...",2993.849315,346126.649183,2832.0,72,6,"61,8","5,5","7,6",3,"45,8",66.0,"0,5",2.2,3.7,"0,5",68,0.2,3,37.04
3,111,Binnenstad,1,Stadsdeel Centrum,"{""coordinates"": [[[5.486100329477308, 51.43455...",3583.765529,665717.709441,4108.0,76,5,"56,8","5,4","7,5",7,"40,3",272.0,"0,9",1.7,3.8,"1,1",74,0.2,4,34.92
4,215,Joriskwartier,2,Stadsdeel Stratum,"{""coordinates"": [[[5.4867601412710165, 51.4287...",2008.871505,148966.813518,1275.0,64,6,"43,4","6,1","7,7",6,"40,7",13.0,"0,9",2.9,2.7,"0,9",47,0.3,5,37.37


In [123]:
dataset["Population density rank"] = dataset["number of residents"] / dataset["GeoshapeArea"]
dataset["Population density rank"] = dataset["Population density rank"].rank(method='first', ascending=False).astype(int)
dataset['AvgIncome'] = dataset['AvgIncome'].str.replace('-', "-1")
dataset['PctHighEducation'] = dataset['PctHighEducation'].str.replace('-', "-1")
dataset['AvgIncome'] = dataset['AvgIncome'].replace('.', -1)
dataset['ScoreGoodLife'] = dataset['ScoreGoodLife'].replace('.', -1)
dataset['PctHighEducation'] = dataset['PctHighEducation'].replace('.', -1)
dataset['AvgIncome'] = dataset['AvgIncome'].str.replace(',', ".")
dataset['ScoreGoodLife'] = dataset['ScoreGoodLife'].str.replace(',', ".")
dataset['PctHighEducation'] = dataset['PctHighEducation'].str.replace(',', ".")

In [124]:
dataset['AvgIncome'] = pd.to_numeric(dataset['AvgIncome'], errors='coerce')
dataset['PctHighEducation'] = pd.to_numeric(dataset['PctHighEducation'], errors='coerce')
dataset['ScoreGoodLife'] = pd.to_numeric(dataset['ScoreGoodLife'], errors='coerce')
dataset = dataset.fillna(-1)
dataset["High education rank"] = dataset["PctHighEducation"].rank(method='first', ascending=False).astype(int)
dataset["Income rank"] = dataset["AvgIncome"].rank(method='first', ascending=False).astype(int)
dataset["Happiness"] = dataset["ScoreGoodLife"].rank(method='first', ascending=False).astype(int)
dataset["Geoshape"] = dataset["Geoshape"].str.replace('""', '"')
dataset.head()

,NbId,NbName,DistrictId,DistrictName,Geoshape,GeoshapePerimeter,GeoshapeArea,number of residents,PctComplainsNoise,PctUnhappy,PctHighEducation,ScoreSocialCohesion,ScoreGoodLife,PctUnemployed,AvgIncome,NumberShops,Distance2Childcare,Distance2Trainstation,Distance2HighwayEntrypoint,Distance2FamilyDoctor,Infrastructure availability,Distance2Сafe,Shops and entertainment,average age,Population density rank,High education rank,Income rank,Happiness
0,216,Bloemenplein,2,Stadsdeel Stratum,"{""coordinates"": [[[5.487913334889891, 51.42373...",1695.060044,133534.195474,1258.0,51,9,36.6,"6,0",7.2,8,36.0,23.0,"0,9",2.9,2.5,"0,8",25,0.3,1,37.42,8,59,69,67
1,236,Nieuwe Erven,2,Stadsdeel Stratum,"{""coordinates"": [[[5.492793179430379, 51.42417...",1635.447970,151145.626948,1136.0,44,9,31.3,"6,4",8.0,10,31.6,22.0,"0,3",3.6,2.2,"0,6",40,0.2,2,39.26,20,74,86,19
2,112,Bergen,1,Stadsdeel Centrum,"{""coordinates"": [[[5.473622298188002, 51.43419...",2993.849315,346126.649183,2832.0,72,6,61.8,"5,5",7.6,3,45.8,66.0,"0,5",2.2,3.7,"0,5",68,0.2,3,37.04,15,17,35,47
3,111,Binnenstad,1,Stadsdeel Centrum,"{""coordinates"": [[[5.486100329477308, 51.43455...",3583.765529,665717.709441,4108.0,76,5,56.8,"5,4",7.5,7,40.3,272.0,"0,9",1.7,3.8,"1,1",74,0.2,4,34.92,38,23,49,55
4,215,Joriskwartier,2,Stadsdeel Stratum,"{""coordinates"": [[[5.4867601412710165, 51.4287...",2008.871505,148966.813518,1275.0,64,6,43.4,"6,1",7.7,6,40.7,13.0,"0,9",2.9,2.7,"0,9",47,0.3,5,37.37,12,45,48,34


In [125]:
tp_df = pd.read_csv("data/extra_dataset_tim_2022.csv", sep=";")
tp_df = tp_df.replace('.', 0)
tp_df = tp_df.fillna(0)
tp_df["doet aan sport %|2022"] = pd.to_numeric(tp_df["doet aan sport %|2022"], errors='coerce')
tp_df["Sport ranking"] = tp_df["doet aan sport %|2022"].astype(float).rank(method='first', ascending=False).astype(int)
tp_df = tp_df.sort_values(by='Sport ranking', ascending=True)
tp_df['Sport ranking'] = tp_df.apply(lambda item: "N/A" if item["doet aan sport %|2022"] == 0 else item["Sport ranking"], axis=1)
tp_df.head()

,Buurten,doet aan sport %|2022,verplaatst zich meestal te voet %|2022,verplaatst zich meestal per fiets %|2022,verplaatst zich meestal per auto %|2022,verplaatst zich meestal per openbaar vervoer %|2022,Sport ranking
94,Strijp S,86,20,54,21,4,1
54,Karpen,85,19,47,31,3,2
57,Koudenhoven,85,19,47,31,3,3
104,Villapark,85,19,47,31,3,4
24,Eikenburg,84,5,63,30,3,5


In [126]:
conditions = [
    tp_df['verplaatst zich meestal te voet %|2022'] > tp_df[['verplaatst zich meestal per fiets %|2022', 'verplaatst zich meestal per auto %|2022', 'verplaatst zich meestal per openbaar vervoer %|2022']].max(axis=1),
    tp_df['verplaatst zich meestal per fiets %|2022'] > tp_df[['verplaatst zich meestal te voet %|2022', 'verplaatst zich meestal per auto %|2022', 'verplaatst zich meestal per openbaar vervoer %|2022']].max(axis=1),
    tp_df['verplaatst zich meestal per auto %|2022'] > tp_df[['verplaatst zich meestal te voet %|2022', 'verplaatst zich meestal per fiets %|2022', 'verplaatst zich meestal per openbaar vervoer %|2022']].max(axis=1),
    tp_df['verplaatst zich meestal per openbaar vervoer %|2022'] > tp_df[['verplaatst zich meestal te voet %|2022', 'verplaatst zich meestal per fiets %|2022', 'verplaatst zich meestal per auto %|2022']].max(axis=1)
]

# Define the corresponding choices
choices = ['foot', 'bike', 'car', 'bus']

# Apply the conditions and choices
tp_df['Preferrable type of transport'] = np.select(conditions, choices, default="car")

columns_to_drop = [
    'verplaatst zich meestal te voet %|2022',
    'verplaatst zich meestal per fiets %|2022',
    'verplaatst zich meestal per auto %|2022',
    'verplaatst zich meestal per openbaar vervoer %|2022'
]

# Drop the specified columns
tp_df.drop(columns=columns_to_drop, inplace=True)
tp_df.rename(columns={'Buurten': 'NbName'}, inplace=True)
tp_df.rename(columns={'doet aan sport %|2022': 'PctSports'}, inplace=True)
dataset = pd.merge(dataset, tp_df, on='NbName', how='inner')
dataset.head(15)

,NbId,NbName,DistrictId,DistrictName,Geoshape,GeoshapePerimeter,GeoshapeArea,number of residents,PctComplainsNoise,PctUnhappy,PctHighEducation,ScoreSocialCohesion,ScoreGoodLife,PctUnemployed,AvgIncome,NumberShops,Distance2Childcare,Distance2Trainstation,Distance2HighwayEntrypoint,Distance2FamilyDoctor,Infrastructure availability,Distance2Сafe,Shops and entertainment,average age,Population density rank,High education rank,Income rank,Happiness,PctSports,Sport ranking,Preferrable type of transport
0,216,Bloemenplein,2,Stadsdeel Stratum,"{""coordinates"": [[[5.487913334889891, 51.42373...",1695.060044,133534.195474,1258.0,51,9,36.6,"6,0",7.2,8,36.0,23.0,"0,9",2.9,2.5,"0,8",25,0.3,1,37.42,8,59,69,67,77,36,bike
1,236,Nieuwe Erven,2,Stadsdeel Stratum,"{""coordinates"": [[[5.492793179430379, 51.42417...",1635.447970,151145.626948,1136.0,44,9,31.3,"6,4",8.0,10,31.6,22.0,"0,3",3.6,2.2,"0,6",40,0.2,2,39.26,20,74,86,19,76,40,bike
2,112,Bergen,1,Stadsdeel Centrum,"{""coordinates"": [[[5.473622298188002, 51.43419...",2993.849315,346126.649183,2832.0,72,6,61.8,"5,5",7.6,3,45.8,66.0,"0,5",2.2,3.7,"0,5",68,0.2,3,37.04,15,17,35,47,81,18,foot
3,111,Binnenstad,1,Stadsdeel Centrum,"{""coordinates"": [[[5.486100329477308, 51.43455...",3583.765529,665717.709441,4108.0,76,5,56.8,"5,4",7.5,7,40.3,272.0,"0,9",1.7,3.8,"1,1",74,0.2,4,34.92,38,23,49,55,75,42,bus
4,215,Joriskwartier,2,Stadsdeel Stratum,"{""coordinates"": [[[5.4867601412710165, 51.4287...",2008.871505,148966.813518,1275.0,64,6,43.4,"6,1",7.7,6,40.7,13.0,"0,9",2.9,2.7,"0,9",47,0.3,5,37.37,12,45,48,34,80,24,bike
5,221,Kerstroosplein,2,Stadsdeel Stratum,"{""coordinates"": [[[5.493297807909147, 51.42352...",2160.569729,208920.876883,1932.0,51,9,37.2,"6,0",7.2,10,33.9,12.0,"0,3",3.9,1.8,"1,0",34,0.2,6,37.3,10,57,77,68,77,37,bike
6,217,Looiakkers,2,Stadsdeel Stratum,"{""coordinates"": [[[5.484935423998946, 51.42964...",1958.395207,227728.828829,699.0,64,6,68.9,"6,1",7.7,3,42.7,16.0,"1,1",3.0,2.7,"0,9",45,0.4,7,39.27,73,5,41,35,80,25,bike
7,212,Rochusbuurt,2,Stadsdeel Stratum,"{""coordinates"": [[[5.493016993060386, 51.43220...",1716.797826,146625.289107,1807.0,64,6,59.5,"6,1",7.7,6,39.0,14.0,"0,3",2.2,3.3,"0,5",53,0.3,8,34.28,2,20,53,36,80,27,bike
8,113,Witte Dame,1,Stadsdeel Centrum,"{""coordinates"": [[[5.469672232324015, 51.44168...",1665.759571,183780.708414,2270.0,72,6,67.7,"5,5",7.6,2,45.1,13.0,"0,9",1.4,3.4,"1,0",42,0.1,9,35.87,1,6,36,48,81,22,foot
9,616,Strijp S,6,Stadsdeel Strijp,"{""coordinates"": [[[5.4611007030010486, 51.4435...",2222.100024,298310.541258,3010.0,65,9,56.2,"6,1",8.0,6,38.6,35.0,"1,1",1.0,2.2,"1,1",3,0.2,10,31.25,7,26,56,20,86,1,bike


In [127]:
dataset = dataset.sort_values(by='ScoreGoodLife', ascending=False)
dataset = dataset.astype(str)
dataset = dataset.replace("-1.0", "N/A")
dataset = dataset.replace(".", "N/A")
dataset = dataset.replace("-", "N/A")
dataset["ScoreGoodLife"] = dataset["ScoreGoodLife"].replace("N/A", np.nan)

In [130]:
dataset.head(15)

,NbId,NbName,DistrictId,DistrictName,Geoshape,GeoshapePerimeter,GeoshapeArea,number of residents,PctComplainsNoise,PctUnhappy,PctHighEducation,ScoreSocialCohesion,ScoreGoodLife,PctUnemployed,AvgIncome,NumberShops,Distance2Childcare,Distance2Trainstation,Distance2HighwayEntrypoint,Distance2FamilyDoctor,Infrastructure availability,Distance2Сafe,Shops and entertainment,average age,Population density rank,High education rank,Income rank,Happiness,PctSports,Sport ranking,Preferrable type of transport
103,542,Luytelaer,5,Stadsdeel Woensel-Noord,"{""coordinates"": [[[5.498408503720958, 51.47109...",3779.90380528896,616939.115506225,981.0,33,4,69.4,"7,3",8.5,N/A,77.9,0.0,"0,6",4.1,2.6,"0,7",87,0.8,104,51.3,84,4,6,2,82,15,bus
99,435,Oude Gracht-Oost,4,Stadsdeel Woensel-Zuid,"{""coordinates"": [[[5.49836882213043, 51.458588...",3573.3893704189804,599451.252747135,1315.0,33,4,63.8,"7,3",8.5,2,63.9,0.0,"0,6",3.1,3.1,"1,0",80,0.7,100,45.05,79,11,18,1,82,16,bus
73,224,Roosten,2,Stadsdeel Stratum,"{""coordinates"": [[[5.491248465498667, 51.41488...",4848.38398289867,899101.67275165,744.0,44,3,N/A,"7,6",8.4,N/A,89.4,0.0,"0,7",4.7,1.8,"1,6",63,0.7,74,41.99,93,102,4,8,84,9,bike
66,225,Eikenburg,2,Stadsdeel Stratum,"{""coordinates"": [[[5.481930065001424, 51.41045...",3891.1638294768,780950.421790145,1540.0,44,3,48.7,"7,6",8.4,4,74.2,0.0,"0,7",4.4,1.4,"1,1",27,1.0,67,44.24,81,37,7,7,84,5,bike
62,218,Elzent-Zuid,2,Stadsdeel Stratum,"{""coordinates"": [[[5.4785751464486205, 51.4292...",1587.63915356652,138355.42197937,299.0,47,4,47.8,"6,7",8.4,N/A,111.0,0.0,"1,2",3.0,2.9,"1,0",65,0.9,63,49.76,80,39,2,6,84,7,bike
59,625,Zwaanstraat,6,Stadsdeel Strijp,"{""coordinates"": [[[5.451767570020675, 51.44794...",3249.67742789912,524900.387142225,1367.0,40,1,83.9,"7,4",8.4,2,72.3,0.0,"0,7",1.6,2.2,"1,5",7,0.6,60,31.63,77,1,11,5,83,10,bike
51,223,Genneperzijde,2,Stadsdeel Stratum,"{""coordinates"": [[[5.484049451030005, 51.42404...",3004.7559120046303,305246.069008415,1351.0,47,4,54.3,"6,7",8.4,2,53.6,0.0,"0,4",3.6,1.8,"0,5",20,0.8,52,54.1,59,29,27,4,84,8,bike
19,213,Elzent-Noord,2,Stadsdeel Stratum,"{""coordinates"": [[[5.484935423998946, 51.42964...",1912.03171933185,211974.88717574,1115.0,47,4,55.4,"6,7",8.4,4,70.7,6.0,"0,7",2.4,3.2,"0,3",59,0.5,20,41.05,47,28,12,3,84,6,bike
70,238,Schuttersbosch,2,Stadsdeel Stratum,"{""coordinates"": [[[5.504236110535605, 51.41935...",2989.5336237535103,384349.463342765,668.0,24,3,64.3,"6,9",8.3,2,56.8,0.0,"1,1",4.6,2.0,"1,3",69,0.6,71,49.51,83,10,22,10,81,21,foot
79,533,Blixembosch-West,5,Stadsdeel Woensel-Noord,"{""coordinates"": [[[5.459703993418563, 51.48346...",4033.59685387676,731655.934964225,2036.0,45,3,50.6,"6,6",8.3,2,68.2,1.0,"0,4",5.1,2.7,"0,9",102,1.0,80,44.45,75,35,15,12,79,30,foot


In [129]:
dataset.to_csv("data/tim_main_dataset.csv")